In [13]:
import pandas as pd
from statistics import median
import csv

In [14]:
project_list = [
    'ruby.csv',
    'metasploit-framework.csv',
    'graylog2-server.csv',
    'android.csv',
    'vagrant.csv',
    'gradle.csv',
    'puppet.csv',
    'opal.csv',
    'rspec-core.csv',
    'rails.csv',
    'okhttp.csv',
    'cloudify.csv',
]

In [15]:
r_file_name = 'new_dynamic_batching.csv'
result_file = open(r_file_name, 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'builds_saved', 'median_delay', 'testall_size']
writer = csv.writer(result_file)
writer.writerow(result_headers)

69

In [16]:
batch_total = 0

In [17]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [18]:
def batchstop4(commits):
    global batch_total
    
    l = len(commits)//2
    
    batch_total += 1        
    
    if 0 in commits:
        
        if len(commits) == 4:
            batch_total += 4
            return commits[0] & commits[1] & commits[2] & commits[3]
        
        return batchbisect(commits[:l]) & batchbisect(commits[l:])
    
    return 1

In [19]:
def batchbisect(commits):
    global batch_total
    
    l = len(commits)//2
    
    batch_total += 1
    if len(commits) == 1:
        return commits[0]
    
    if 0 in commits:
        return batchbisect(commits[:l]) & batchbisect(commits[l:])
    
    return 1

In [20]:
def batchbisect_dynamic(commits):
    global batch_total
    
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    
    while i < length:
        batch = commits[i:i+batch_size]
        fails_per = 100*batch.count(0)/batch_size
        
        delays.extend([batch_size-x-1 for x in range(batch_size)])
        
        batch_total = 0
        res = batchbisect(batch)
        builds += batch_total
        print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))
        
        i += batch_size
        
        if res == 0:
            
            if fails_per > 50 & batch[-1] == 0:
                #if batch has a lot of failures then definitely split
                
                if batch_size == 1:
                    continue
                else:
                    batch_size //= 2
            
            if fails_per < 20 & batch[-1] == 1:
                continue
        
        else:
            if batch_size == 16:
                continue
            else:
                batch_size *= 2
    
    return builds, delays
            

In [21]:
def batchstop4_dynamic(commits):
    global batch_total
    
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    
    while i < length:
        batch = commits[i:i+batch_size]
        fails_per = 100*batch.count(0)/batch_size
        delays.extend([batch_size-x-1 for x in range(batch_size)])
        
        batch_total = 0
        res = batchstop4(batch)
        builds += batch_total
        print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))
        
        i += batch_size
        
        if res == 0:
            if fails_per > 50 & batch[-1] == 0:
                #if batch has a lot of failures then definitely split
                
                if batch_size == 4:
                    continue
                else:
                    batch_size //= 2
            
            if fails_per < 20 & batch[-1] == 1:
                continue
        
        else:
            if batch_size == 16:
                continue
            else:
                batch_size *= 2
    
    return builds, delays

In [22]:
for p in project_list:
    data = pd.read_csv('data/extracted_project_travis/'+p)
    if p == 'ruby.csv':
        data = data[ data['git_branch'] == 'trunk']
    else:
        data = data[ data['git_branch'] == 'master']
    data = data.iloc[100:]
    results = output_values(data['tr_status'].tolist())
    builds, delays = batchbisect_dynamic(results)
    reqd = 100-(100*builds/len(results))
    fr = 100*results.count(0)/len(results)
    
    writer.writerow([p, 'BATCHBISECT', '-1', reqd, median(delays), len(results)])
    
    

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 3
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 4
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 6
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 7
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 9
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 18
Building 8 [0, 0, 0, 1, 1, 0, 0, 0] 15 33
Building 4 [0, 0, 0, 1] 7 40
Building 2 [1, 1] 1 41
Building 4 [1, 1, 1, 1] 1 42
Building 8 [1, 1, 1, 1, 0, 0, 1, 1] 7 49
Building 4 [1, 1, 1, 1] 1 50
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 51
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 52
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Building 2 [1, 0] 3 8203
Building 1 [1] 1 8204
Building 2 [0, 1] 3 8207
Building 1 [1] 1 8208
Building 2 [1, 1] 1 8209
Building 4 [1, 1, 1, 1] 1 8210
Building 8 [1, 1, 0, 1, 1, 1, 0, 0] 11 8221
Building 4 [0, 1, 0, 0] 7 8228
Building 2 [1, 1] 1 8229
Building 4 [0, 0, 0, 1] 7 8236
Building 2 [1, 1] 1 8237
Building 4 [1, 1, 1, 1] 1 8238
Building 8 [1, 0, 1, 0, 1, 1, 1, 0] 13 8251
Building 4 [1, 1, 1, 1] 1 8252
Building 8 [1, 0, 1, 1, 1, 1, 1, 1] 7 8259
Building 4 [1, 1, 1, 1] 1 8260
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 8261
Building 16 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 8270
Building 8 [1, 1, 1, 1, 0, 1, 1, 1] 7 8277
Building 4 [1, 1, 1, 1] 1 8278
Building 8 [1, 1, 1, 1, 1, 1, 1, 0] 7 8285
Building 4 [0, 0, 1, 1] 5 8290
Building 2 [1, 1] 1 8291
Building 4 [1, 1, 1, 1] 1 8292
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 8293
Building 16 [1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 8304
Building 8 [1, 1, 0, 1, 1, 1, 1, 1] 7 8311
Building 4 [1, 1, 1, 1] 1 8312
Building 8 [1

Building 16 [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 9
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 10
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 11
Building 16 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 20
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 21
Building 16 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1] 9 30
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 31
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 32
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1] 9 41
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 42
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 43
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1] 9 52
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 53
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 62
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 63
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 64
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 65
Building 16 [1, 1, 1, 1, 1, 1

Building 16 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 31 31
Building 8 [0, 0, 0, 0, 0, 0, 0, 0] 15 46
Building 4 [0, 0, 0, 0] 7 53
Building 2 [0, 0] 3 56
Building 1 [0] 1 57
Building 1 [0] 1 58
Building 1 [0] 1 59
Building 1 [0] 1 60
Building 1 [0] 1 61
Building 1 [0] 1 62
Building 1 [0] 1 63
Building 1 [0] 1 64
Building 1 [0] 1 65
Building 1 [0] 1 66
Building 1 [0] 1 67
Building 1 [0] 1 68
Building 1 [0] 1 69
Building 1 [0] 1 70
Building 1 [0] 1 71
Building 1 [0] 1 72
Building 1 [0] 1 73
Building 1 [0] 1 74
Building 1 [0] 1 75
Building 1 [0] 1 76
Building 1 [0] 1 77
Building 1 [0] 1 78
Building 1 [0] 1 79
Building 1 [0] 1 80
Building 1 [0] 1 81
Building 1 [0] 1 82
Building 1 [0] 1 83
Building 1 [0] 1 84
Building 1 [0] 1 85
Building 1 [0] 1 86
Building 1 [0] 1 87
Building 1 [0] 1 88
Building 1 [0] 1 89
Building 1 [0] 1 90
Building 1 [0] 1 91
Building 1 [0] 1 92
Building 1 [0] 1 93
Building 1 [0] 1 94
Building 1 [0] 1 95
Building 1 [0] 1 96
Building 1 [0] 1 97
Building 1 [0] 1 98

Building 1 [0] 1 8333
Building 1 [0] 1 8334
Building 1 [0] 1 8335
Building 1 [0] 1 8336
Building 1 [0] 1 8337
Building 1 [0] 1 8338
Building 1 [0] 1 8339
Building 1 [0] 1 8340
Building 1 [1] 1 8341
Building 2 [1, 1] 1 8342
Building 4 [1, 0, 1, 1] 5 8347
Building 2 [1, 1] 1 8348
Building 4 [1, 1, 1, 1] 1 8349
Building 8 [1, 0, 1, 1, 1, 1, 1, 1] 7 8356
Building 4 [1, 1, 1, 1] 1 8357
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 8358
Building 16 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1] 15 8373
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 8374
Building 16 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 8385
Building 8 [1, 1, 1, 1, 1, 0, 1, 1] 7 8392
Building 4 [1, 1, 1, 0] 5 8397
Building 2 [1, 1] 1 8398
Building 4 [1, 1, 1, 1] 1 8399
Building 8 [1, 1, 1, 1, 1, 0, 1, 1] 7 8406
Building 4 [0, 1, 1, 1] 5 8411
Building 2 [0, 0] 3 8414
Building 1 [0] 1 8415
Building 1 [0] 1 8416
Building 1 [0] 1 8417
Building 1 [0] 1 8418
Building 1 [0] 1 8419
Building 1 [0] 1 8420
Building 1 [0] 1 8421
Building 

Building 1 [1] 1 1261
Building 2 [1, 1] 1 1262
Building 4 [1, 0, 1, 0] 7 1269
Building 2 [1, 1] 1 1270
Building 4 [0, 1, 1, 1] 5 1275
Building 2 [1, 1] 1 1276
Building 4 [1, 0, 1, 0] 7 1283
Building 2 [1, 1] 1 1284
Building 4 [1, 1, 1, 0] 5 1289
Building 2 [0, 1] 3 1292
Building 1 [0] 1 1293
Building 1 [0] 1 1294
Building 1 [0] 1 1295
Building 1 [1] 1 1296
Building 2 [1, 0] 3 1299
Building 1 [1] 1 1300
Building 2 [1, 0] 3 1303
Building 1 [0] 1 1304
Building 1 [0] 1 1305
Building 1 [1] 1 1306
Building 2 [1, 1] 1 1307
Building 4 [1, 0, 0, 1] 7 1314
Building 2 [0, 1] 3 1317
Building 1 [1] 1 1318
Building 2 [1, 1] 1 1319
Building 4 [0, 0, 0, 0] 7 1326
Building 2 [1, 1] 1 1327
Building 4 [1, 1, 0, 1] 5 1332
Building 2 [1, 1] 1 1333
Building 4 [1, 0, 1, 0] 7 1340
Building 2 [0, 0] 3 1343
Building 1 [1] 1 1344
Building 2 [0, 1] 3 1347
Building 1 [0] 1 1348
Building 1 [0] 1 1349
Building 1 [0] 1 1350
Building 1 [1] 1 1351
Building 2 [1, 1] 1 1352
Building 4 [1, 1, 0, 1] 5 1357
Building 2 [0, 1

In [23]:
for p in project_list:
    data = pd.read_csv('data/extracted_project_travis/'+p)
    if p == 'ruby.csv':
        data = data[ data['git_branch'] == 'trunk']
    else:
        data = data[ data['git_branch'] == 'master']
    data = data.iloc[100:]
    results = output_values(data['tr_status'].tolist())
    builds, delays = batchstop4_dynamic(results)
    reqd = 100-(100*builds/len(results))
    fr = 100*results.count(0)/len(results)
    
    writer.writerow([p, 'BATCHSTOP4', '-1', reqd, median(delays), len(results)])
    
    #print('{:>40} {:>10} {:>30} {:>10}'.format(p, len(results), reqd, median(delays)))

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 3
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 4
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 5
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 6
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 7
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 8
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 9
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 18
Building 8 [0, 0, 0, 1, 1, 0, 0, 0] 15 33
Building 4 [0, 0, 0, 1] 5 38
Building 4 [1, 1, 1, 1] 1 39
Building 8 [1, 1, 1, 1, 1, 1, 0, 0] 7 46
Building 4 [1, 1, 1, 1] 1 47
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 48
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 49
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 50
Buildi

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 10
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 11
Building 16 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0] 17 28
Building 8 [0, 0, 1, 1, 1, 0, 1, 1] 11 39
Building 4 [1, 1, 1, 1] 1 40
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 41
Building 16 [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1] 9 50
Building 8 [0, 0, 1, 1, 1, 1, 1, 1] 7 57
Building 4 [1, 1, 1, 1] 1 58
Building 8 [1, 1, 0, 0, 0, 0, 0, 1] 13 71
Building 4 [1, 1, 0, 1] 5 76
Building 4 [1, 1, 1, 1] 1 77
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 78
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1] 9 87
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 88
Building 16 [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1] 17 105
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 106
Building 16 [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 117
Building 8 [1, 1, 1, 1, 0, 0, 1, 1] 7 124
Building 4 [1, 1, 1, 1] 1 125
Building 8 [1, 

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0] 19 20
Building 8 [1, 1, 1, 1, 1, 0, 0, 0] 9 29
Building 4 [0, 1, 1, 1] 5 34
Building 4 [0, 1, 1, 1] 5 39
Building 4 [1, 1, 1, 1] 1 40
Building 8 [1, 1, 1, 1, 0, 1, 1, 1] 7 47
Building 4 [1, 1, 1, 1] 1 48
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 49
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0] 15 64
Building 8 [1, 1, 1, 1, 1, 0, 0, 0] 9 73
Building 4 [0, 0, 0, 0] 5 78
Building 4 [0, 0, 1, 0] 5 83
Building 4 [0, 1, 1, 1] 5 88
Building 4 [0, 0, 1, 0] 5 93
Building 4 [0, 1, 1, 1] 5 98
Building 4 [1, 0, 0, 1] 5 103
Building 4 [1, 1, 1, 0] 5 108
Building 4 [0, 1, 1, 1] 5 113
Building 4 [1, 1, 0, 1] 5 118
Building 4 [1, 1, 0, 0] 5 123
Building 4 [0, 1, 1, 1] 5 128
Building 4 [1, 1, 0, 0] 5 133
Building 4 [1, 0, 1, 1] 5 138
Building 4 [1, 1, 1, 1] 1 139
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 140
Building 16 [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1] 17 

Building 4 [1, 1, 1, 1] 1 6672
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 6673
Building 16 [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0] 25 6698
Building 8 [1, 1, 1, 1, 1, 1, 0, 1] 7 6705
Building 4 [0, 1, 1, 1] 5 6710
Building 4 [1, 1, 1, 1] 1 6711
Building 8 [1, 1, 1, 1, 1, 0, 1, 1] 7 6718
Building 4 [0, 1, 1, 1] 5 6723
Building 4 [1, 1, 1, 1] 1 6724
Building 8 [1, 0, 1, 1, 0, 1, 1, 1] 11 6735
Building 4 [0, 0, 1, 1] 5 6740
Building 4 [1, 1, 1, 1] 1 6741
Building 8 [1, 1, 0, 1, 1, 1, 0, 1] 11 6752
Building 4 [0, 1, 0, 1] 5 6757
Building 4 [1, 1, 0, 0] 5 6762
Building 4 [1, 0, 1, 1] 5 6767
Building 4 [1, 0, 0, 1] 5 6772
Building 4 [0, 0, 1, 1] 5 6777
Building 4 [1, 1, 0, 0] 5 6782
Building 4 [0, 0, 1, 0] 5 6787
Building 4 [1, 1, 1, 1] 1 6788
Building 8 [1, 0, 1, 0, 1, 1, 0, 1] 13 6801
Building 4 [0, 1, 1, 1] 5 6806
Building 4 [1, 1, 1, 1] 1 6807
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 6808
Building 16 [0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 15 6823
Building 8 [1, 1, 1, 1, 0, 0, 1

Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 2716
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2717
Building 16 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 31 2748
Building 8 [0, 1, 0, 1, 0, 1, 1, 1] 13 2761
Building 4 [1, 1, 1, 1] 1 2762
Building 8 [1, 1, 1, 1, 1, 0, 0, 1] 9 2771
Building 4 [0, 1, 0, 1] 5 2776
Building 4 [1, 1, 1, 1] 1 2777
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 2778
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2779
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1] 9 2788
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 2789
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2790
Building 16 [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 2801
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 2802
Building 16 [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1] 19 2821
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 2822
Building 16 [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1] 17 2839
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 2840
Building 16 [1